In [51]:
import numpy as np
import pandas as pd
import os
import json
from pandas.io.json import json_normalize
from sklearn import metrics
from catboost import Pool,CatBoostRegressor,CatBoostClassifier
import random
import time
import datetime
patch="D:\Python\\Google"
def run_lgb(train_x, train_y, val_x, val_y,val_id):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "learning_rate" : 0.02,
        'n_estimators': 50,
         'num_iterations': 50,
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(50/3), verbose_eval=100)
    
    pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_id.values})
    val_pred_df["transactionRevenue"] = val_y
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].mean().reset_index()
#    return model
    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))
def run_cat(train_x, train_y, val_x, val_y,val_id):
    pool=Pool(train_x,train_y)
    val_pool=Pool(val_x,val_y)
    model = CatBoostRegressor(iterations=50, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    
    pred_val = model.predict(val_x)
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_id.values})
    val_pred_df["transactionRevenue"] = val_y
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].mean().reset_index()
#    return model
    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))

def run_cat_bin(train_x, train_y, val_x, val_y,val_id):
    pool=Pool(train_x,train_y)
    val_pool=Pool(val_x,val_y)
    model = CatBoostClassifier(iterations=80, learning_rate=0.05,loss_function='CrossEntropy', eval_metric='AUC')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    pred_train=model.predict(train_x, prediction_type="Probability")[:,1]
    pred_val = model.predict(val_x, prediction_type="Probability")[:,1]
#    return val_pred_df
    return  pred_train,pred_val

def run_lgb_bin(train_x, train_y, val_x, val_y,val_id):
    params = {
        "objective" : "binary",
        "metric" : "auc", 
        "learning_rate" : 0.02,
        'n_estimators': 50,
         'num_iterations': 50,
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(50/3), verbose_eval=100)
    
    pred_train = model.predict(train_x, num_iteration=model.best_iteration)
    pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    
#    return model
    return  pred_train,pred_val

    
def multimodel(train_x, train_y, val_x, val_y):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "learning_rate" : 0.01,
        'n_estimators': 50,
         'num_iterations': 70,
    }
    i=random.randint(0,99999)
    train1=pd.concat([train_x.reset_index(drop=True),pd.DataFrame(train_y,columns=['bg'])],axis=1)
    train_x_bad=train1[train1['bg']>0].drop(['bg'],axis=1)
    train_y_bad=train1[train1['bg']>0]['bg'].values
    train_x_good=train1[train1['bg']==0].drop(['bg'],axis=1).sample(len(train_x_bad)*6,random_state=i)
    train_y_good=train1[train1['bg']==0]['bg'].sample(len(train_x_bad)*6,random_state=i).values
    
    val1=pd.concat([val_x.reset_index(drop=True),pd.DataFrame(val_y,columns=['bg'])],axis=1)
    val_x_bad=val1[val1['bg']>0].drop(['bg'],axis=1)
    val_y_bad=val1[val1['bg']>0]['bg'].values
    val_x_good=val1[val1['bg']==0].drop(['bg'],axis=1).sample(len(val_x_bad)*6,random_state=i)
    val_y_good=val1[val1['bg']==0]['bg'].sample(len(val_x_bad)*6,random_state=i).values
    
    train_x,train_y=pd.concat([train_x_bad,train_x_good]),np.concatenate([train_y_bad,train_y_good])
    val_x,val_y=pd.concat([val_x_bad,val_x_good]),np.concatenate([val_y_bad,val_y_good])
#     pool=Pool(train_x,train_y)
#     val_pool=Pool(val_x,val_y)
#     model = CatBoostRegressor(iterations=50, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
#     model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(50/3), verbose_eval=100)
    return model

def run_multimodel(train_x, train_y, val_x, val_y,val_id):
    result_val=[]
    for i in range(0,10):
        model=multimodel(train_x, train_y, val_x, val_y)
        result_val.append(model.predict(val_x, num_iteration=model.best_iteration))
    pred_val = sum(result_val)/float(len(result_val))
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"fullVisitorId":val_id.values})
    val_pred_df["transactionRevenue"] = val_y
    val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
    val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
#    return val_pred_df
    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))
def run_lgb2(train_x, train_y, val_x, val_y,val_id):
    params = {
        "objective" : "regression",
        "metric" : "rmse", 
        "learning_rate" : 0.02,
        'n_estimators': 50,
         'num_iterations': 50,
    }
    
    lgtrain = lgb.Dataset(train_x, label=train_y)
    lgval = lgb.Dataset(val_x, label=val_y)
    model = lgb.train(params, lgtrain, valid_sets=[lgval], early_stopping_rounds=int(itr/3), verbose_eval=100)
    
    pred_val = model.predict(val_x, num_iteration=model.best_iteration)
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"transactionRevenue":val_y,"PredictedRevenue":pred_val})
#    return model
    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))
def run_cat2(train_x, train_y, val_x, val_y,val_id):
    pool=Pool(train_x,train_y)
    val_pool=Pool(val_x,val_y)
    model = CatBoostRegressor(iterations=30, learning_rate=0.01,loss_function='RMSE', eval_metric='RMSE')
    model.fit(pool,eval_set=val_pool,use_best_model=True,verbose_eval=False)
    
    pred_val = model.predict(val_x)
    pred_val[pred_val<0] = 0
    val_pred_df = pd.DataFrame({"transactionRevenue":val_y,"PredictedRevenue":pred_val})
    return model
#    return  np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values)))


In [8]:
train=pd.read_pickle(os.path.join(patch,'train_v2_clean.pkl'))
test=pd.read_pickle(os.path.join(patch,'test_v2_clean.pkl'))
train=train.append(test,ignore_index=True)

In [9]:
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

#train=grouping_month(train)
train["totals.totalTransactionRevenue"].fillna(0, inplace=True)
train["visitStartTime"] = pd.to_datetime(train["visitStartTime"],unit='s')

train['day']=train['visitStartTime'].dt.day.astype('uint8')
train['weekday']=train['visitStartTime'].apply(lambda x:x.date().weekday())
train['minute']=train['visitStartTime'].dt.minute.astype('uint8')
train['hour']=train['visitStartTime'].dt.hour.astype('uint8')

# label encode the categorical variables and convert the numerical variables to float
#ar=df['geoNetwork.networkDomain'].value_counts()[df['geoNetwork.networkDomain'].value_counts()<=4].index
#train['geoNetwork.networkDomain']=['some unfamous domain' if x in ar else x for x in df['geoNetwork.networkDomain'].values]
#del ar
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect',
           'customDimensions_value','weekday']
for col in cat_cols:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values.astype('str')))
    train[col] = lbl.transform(list(train[col].values.astype('str')))
#train=pd.get_dummies(train,columns=cat_cols,prefix=cat_cols,prefix_sep='_')

num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits',
            'totals.transactions','totals.sessionQualityDim','totals.timeOnSite','day','minute','hour']

# num_cols = ["totals.hits", "totals.pageviews", "visitNumber", 'totals.bounces',  'totals.newVisits',
#            'totals.transactions','totals.sessionQualityDim','totals.timeOnSite','day','minute','hour',
#             'totals.totalTransactionRevenue_month_sum',
#            'totals.totalTransactionRevenue_month_count', 'totals.hits_month_sum',
#            'totals.hits_month_count', 'totals.hits_month_median',
#            'totals.pageviews_month_sum', 'totals.pageviews_month_mean',
#            'totals.pageviews_month_median', 'totals.bounces_month_sum',
#            'totals.bounces_month_mean', 'totals.bounces_month_median',
#            'totals.newVisits_month_sum', 'totals.newVisits_month_mean',
#            'totals.newVisits_month_median']    
for col in num_cols:
    train[col] = train[col].astype(float)
    
train['totals.totalTransactionRevenue'] = train['totals.totalTransactionRevenue'].astype(float)
train['totals.totalTransactionCount'] = np.where(train['totals.totalTransactionRevenue'].values>0,1,0)
train['device.isMobile']=train['device.isMobile'].astype(int)

# Split the train dataset into development and valid based on time 
visitid=train['fullVisitorId']
train=train.drop(['date','visitId','totals.transactionRevenue'],axis=1)

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect
customDimensions_value
weekday


In [69]:
#train["date"]=pd.to_datetime(train["date"], format='%Y%m%d')
#train['Month']=train["date"].dt.to_period("M")
def grouping_month(df):
    df["date"]=pd.to_datetime(df["visitStartTime"], format='%Y%m%d')
    df['Month']=df["date"].dt.to_period("M")
    df['totals.hits']=df['totals.hits'].astype(int)
    df['totals.newVisits'].fillna(0, inplace=True)
    df['totals.newVisits']=df['totals.newVisits'].astype(int)
    df['totals.pageviews'].fillna(0, inplace=True)
    df['totals.pageviews']=df['totals.pageviews'].astype(int)
    df['totals.bounces'].fillna(0, inplace=True)
    df['totals.bounces']=df['totals.bounces'].astype(int)
    df['totals.totalTransactionRevenue'].fillna(0, inplace=True)
    df['totals.totalTransactionRevenue']=df['totals.totalTransactionRevenue'].astype(float)
    aggs={
        'totals.totalTransactionRevenue': ['sum','count'],
        'totals.hits': ['sum','count', 'median'],
        'totals.pageviews': ['sum','mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }
    temp=df.groupby(['fullVisitorId','Month'],as_index=False).agg({**aggs})
    col=["_month_".join(x) for x in temp.columns.ravel()[2:]]
    col.insert(0,'Month')
    col.insert(0,'fullVisitorId')
    temp.columns=col
    #temp.columns=['fullVisitorId','Month','transaction_month_count']
    df=pd.merge(df,temp,on=['fullVisitorId','Month'],how='left')
    return df
def grouping_all(df):
    aggs={
        'totals.totalTransactionRevenue': ['sum'],
        'totals.hits': ['sum','count', 'median'],
        'totals.pageviews': ['sum','mean', 'median'],
        'totals.bounces': ['sum', 'mean', 'median'],
        'totals.newVisits': ['sum', 'mean', 'median']
    }
    temp=df.groupby(['fullVisitorId'],as_index=False).agg({**aggs})
    col=["_all_".join(x) for x in temp.columns.ravel()[1:]]
    col.insert(0,'fullVisitorId')
    temp.columns=col
    #temp.columns=['fullVisitorId','Month','transaction_month_count']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    df.fillna(0, inplace=True)
    return df
def grouping_date(df,y,m,d):
    temp=df.groupby('fullVisitorId',as_index=False)['visitStartTime'].max()
    temp.columns=['fullVisitorId','last_visit']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    temp=df[df['totals.totalTransactionRevenue']>0].groupby('fullVisitorId',as_index=False)['visitStartTime'].max()
    temp.columns=['fullVisitorId','last_visit_money']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    dat_cols=['visitStartTime','last_visit','last_visit_money']
    for col in dat_cols:
        df[col] = pd.to_datetime(df[col],unit='s')
    df['last_visit']=df['last_visit']-datetime.datetime(y,m,d)
    df['last_visit']=df['last_visit'].dt.days
    df['last_visit_money']=df['last_visit_money']-datetime.datetime(y,m,d)
    df['last_visit_money']=df['last_visit_money'].dt.days
    return df
def grouping_date(df,y,m,d):
    temp=df.groupby('fullVisitorId',as_index=False)['visitStartTime'].max()
    temp.columns=['fullVisitorId','last_visit']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    temp=df[df['totals.totalTransactionRevenue']>0].groupby('fullVisitorId',as_index=False)['visitStartTime'].max()
    temp.columns=['fullVisitorId','last_visit_money']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    
    temp=df.groupby('fullVisitorId',as_index=False)['visitStartTime'].min()
    temp.columns=['fullVisitorId','duration_visit']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    temp=df[df['totals.totalTransactionRevenue']>0].groupby('fullVisitorId',as_index=False)['visitStartTime'].min()
    temp.columns=['fullVisitorId','duration_visit_money']
    df=pd.merge(df,temp,on='fullVisitorId',how='left')
    
    dat_cols=['visitStartTime','last_visit','last_visit_money','duration_visit','duration_visit_money']
    for col in dat_cols:
        df[col] = pd.to_datetime(df[col],unit='s')
    df['duration_visit']=df['last_visit']-df['duration_visit']
    df['duration_visit']=df['duration_visit'].dt.days
    df['duration_visit_money']=df['last_visit_money']-df['duration_visit_money']
    df['duration_visit_money']=df['duration_visit_money'].dt.days
    df['last_visit']=datetime.datetime(y,m,d)-df['last_visit']
    df['last_visit']=df['last_visit'].dt.days
    df['last_visit_money']=datetime.datetime(y,m,d)-df['last_visit_money']
    df['last_visit_money']=df['last_visit_money'].dt.days
    return df
def grouping_id(df):
    aggs={
        'totals.totalTransactionRevenue': ['sum'],
        'totals.totalTransactionCount': ['sum','mean'],
        'totals.hits': ['count'],
        'totals.pageviews': ['sum', 'median'],
        'totals.bounces': ['sum'],
        'totals.newVisits': ['sum', 'mean'],
        'customDimensions_value': ['max','min'],
        'geoNetwork.subContinent': ['max','min'],
        'geoNetwork.region': ['max','min'],
        'geoNetwork.networkDomain': ['max','min'],
        'geoNetwork.continent': ['max','min'],
        'device.operatingSystem': ['max','min'],
        'device.deviceCategory': ['max','min'],
        'channelGrouping': ['max','min'],
        'visitNumber': ['max'],
        'last_visit_money': ['max'],
         'duration_visit': ['max']
#        'totals.timeOnSite': ['mean'],
#        'hour': ['max','mean','last'],
#        'weekday': ['mean'],
#         'duration_visit_money': ['max']
    }
    temp=df.groupby(['fullVisitorId'],as_index=False).agg({**aggs})
    col=["_".join(x) for x in temp.columns.ravel()[1:]]
    col.insert(0,'fullVisitorId')
    temp.columns=col
    temp.fillna(0, inplace=True)
    return temp

In [22]:
def super_data(train,a1,b1,c1,a2,b2,c2,a3,b3,c3,a4,b4,c4):
    train_x_index=train[(train['visitStartTime']>=datetime.date(a1,b1,c1))&(train['visitStartTime']<datetime.date(a2,b2,c2))].index
    train_y_index=train[(train['visitStartTime']>=datetime.date(a3,b3,c3))&(train['visitStartTime']<datetime.date(a4,b4,c4))].index
    df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
    df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
    train_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[train_x_index],df_train_y,how='left',on='fullVisitorId')
    train_y=np.log1p(train_y['Revenue'].fillna(0).values)
    train_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[train_x_index]
    train_x=grouping_all(train.loc[train_x_index])   
    train_x=train_x.drop(['visitStartTime','fullVisitorId'],axis=1)
    return train_x,train_y
import time
import datetime
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2017,2,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2017,3,1))&(train['visitStartTime']<datetime.date(2017,5,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2017,2,1))&(train['visitStartTime']<datetime.date(2017,8,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2017,9,1))&(train['visitStartTime']<datetime.date(2018,11,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
train_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[train_x_index],df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[train_x_index]
train_x=grouping_all(train.loc[train_x_index])
#train_x=grouping_date(train_x.loc[train_x_index],2016,11,1)
train_x=train_x.drop(['visitStartTime','fullVisitorId'],axis=1)
temp_x,temp_y=super_data(train,2017,3,1,2017,9,1,2017,10,1,2017,12,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)
temp_x,temp_y=super_data(train,2017,10,1,2018,4,1,2018,5,1,2018,7,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
val_y=pd.merge(train.drop(['visitStartTime'],axis=1).loc[val_x_index],df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=train.drop(['visitStartTime','fullVisitorId'],axis=1).loc[val_x_index]
val_x=grouping_all(train.loc[val_x_index])
#val_x=grouping_date(val_x.loc[val_x_index],2017,11,1)
val_x=val_x.drop(['visitStartTime','fullVisitorId'],axis=1)

run_lgb(train_x,train_y,val_x,val_y,val_id)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[50]	valid_0's rmse: 0.890738


0.09722046751876291

In [70]:
def super_data(train,a1,b1,c1,a2,b2,c2,a3,b3,c3,a4,b4,c4):
    train_x_index=train[(train['visitStartTime']>=datetime.date(a1,b1,c1))&(train['visitStartTime']<datetime.date(a2,b2,c2))].index
    train_y_index=train[(train['visitStartTime']>=datetime.date(a3,b3,c3))&(train['visitStartTime']<datetime.date(a4,b4,c4))].index
    df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
    df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
    df_train_x=grouping_id(grouping_date(train.loc[train_x_index],a2,b2,c2))
    train_y=pd.merge(df_train_x,df_train_y,how='left',on='fullVisitorId')
    train_y=np.log1p(train_y['Revenue'].fillna(0).values)
    train_x=df_train_x.drop(['fullVisitorId'],axis=1)
    return train_x,train_y
import time
import datetime
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2017,1,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2017,2,1))&(train['visitStartTime']<datetime.date(2017,4,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2018,2,1))&(train['visitStartTime']<datetime.date(2018,8,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2018,9,1))&(train['visitStartTime']<datetime.date(2018,11,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
df_train_x=grouping_id(grouping_date(train.loc[train_x_index],2017,2,1))
train_y=pd.merge(df_train_x,df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=df_train_x.drop(['fullVisitorId'],axis=1)
#train_x=grouping_date(train_x.loc[train_x_index],2016,11,1)
temp_x,temp_y=super_data(train,2017,1,1,2017,6,1,2017,7,1,2017,9,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)
temp_x,temp_y=super_data(train,2017,6,1,2017,11,1,2017,12,1,2018,2,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
df_val_x=grouping_id(grouping_date(train.loc[val_x_index],2018,8,1))
val_y=pd.merge(df_val_x,df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=df_val_x.drop(['fullVisitorId'],axis=1)

run_lgb2(train_x,train_y,val_x,val_y,val_id)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 16 rounds.
Early stopping, best iteration is:
[34]	valid_0's rmse: 0.32451


0.05435599257575876

In [36]:
def super_data(train,a1,b1,c1,a2,b2,c2,a3,b3,c3,a4,b4,c4):
    train_x_index=train[(train['visitStartTime']>=datetime.date(a1,b1,c1))&(train['visitStartTime']<datetime.date(a2,b2,c2))].index
    train_y_index=train[(train['visitStartTime']>=datetime.date(a3,b3,c3))&(train['visitStartTime']<datetime.date(a4,b4,c4))].index
    df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
    df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
    df_train_x=grouping_id(grouping_date(train.loc[train_x_index],a2,b2,c2))
    train_y=pd.merge(df_train_x,df_train_y,how='left',on='fullVisitorId')
    train_y=np.log1p(train_y['Revenue'].fillna(0).values)
    train_x=df_train_x.drop(['fullVisitorId'],axis=1)
    return train_x,train_y
import time
import datetime
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2017,1,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2017,2,1))&(train['visitStartTime']<datetime.date(2017,4,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2018,2,1))&(train['visitStartTime']<datetime.date(2018,8,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2018,9,1))&(train['visitStartTime']<datetime.date(2018,11,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
df_train_x=grouping_id(grouping_date(train.loc[train_x_index],2017,2,1))
train_y=pd.merge(df_train_x,df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=df_train_x.drop(['fullVisitorId'],axis=1)
#train_x=grouping_date(train_x.loc[train_x_index],2016,11,1)
temp_x,temp_y=super_data(train,2017,1,1,2017,6,1,2017,7,1,2017,9,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)
temp_x,temp_y=super_data(train,2017,6,1,2017,11,1,2017,12,1,2018,2,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
df_val_x=grouping_id(grouping_date(train.loc[val_x_index],2018,8,1))
val_y=pd.merge(df_val_x,df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=df_val_x.drop(['fullVisitorId'],axis=1)

#run_lgb2(train_x,train_y,val_x,val_y,val_id)
result=pd.DataFrame(columns=['Name','Score'])
for col in train_x.columns:
    result.loc[len(result)]=[col,run_lgb2(train_x.drop(col,axis=1),train_y,val_x.drop(col,axis=1),val_y,val_id)]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\lightgbm\engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[48]	valid_0's rmse: 0.323327
Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[43]	valid_0's rmse: 0.324482
Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[38]	valid_0's rmse: 0.324463
Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[45]	valid_0's rmse: 0.324592
Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[50]	valid_0's rmse: 0.324459
Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[41]	valid_0's rmse: 0.324566
Training until validation scores don't improve for 16 rounds.
Did not meet early stopping. Best iteration is:
[44]	valid_0's rmse: 0.324473
Training until valid

In [49]:
def super_data(train,a1,b1,c1,a2,b2,c2,a3,b3,c3,a4,b4,c4):
    train_x_index=train[(train['visitStartTime']>=datetime.date(a1,b1,c1))&(train['visitStartTime']<datetime.date(a2,b2,c2))].index
    train_y_index=train[(train['visitStartTime']>=datetime.date(a3,b3,c3))&(train['visitStartTime']<datetime.date(a4,b4,c4))].index
    df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
    df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
    df_train_x=grouping_id(grouping_date(train.loc[train_x_index],a2,b2,c2))
    train_y=pd.merge(df_train_x,df_train_y,how='left',on='fullVisitorId')
    train_y=np.log1p(train_y['Revenue'].fillna(0).values)
    train_x=df_train_x.drop(['fullVisitorId'],axis=1)
    return train_x,train_y
import time
import datetime
train_x_index=train[(train['visitStartTime']>=datetime.date(2016,8,1))&(train['visitStartTime']<datetime.date(2017,1,1))].index
train_y_index=train[(train['visitStartTime']>=datetime.date(2017,2,1))&(train['visitStartTime']<datetime.date(2017,4,1))].index
val_x_index=train[(train['visitStartTime']>=datetime.date(2018,2,1))&(train['visitStartTime']<datetime.date(2018,8,1))].index
val_y_index=train[(train['visitStartTime']>=datetime.date(2018,9,1))&(train['visitStartTime']<datetime.date(2018,11,1))].index

df_train_y=train.drop(['visitStartTime'],axis=1).loc[train_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_train_y=df_train_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
df_train_x=grouping_id(grouping_date(train.loc[train_x_index],2017,2,1))
train_y=pd.merge(df_train_x,df_train_y,how='left',on='fullVisitorId')
train_y=np.log1p(train_y['Revenue'].fillna(0).values)
train_x=df_train_x.drop(['fullVisitorId'],axis=1)
#train_x=grouping_date(train_x.loc[train_x_index],2016,11,1)
temp_x,temp_y=super_data(train,2017,1,1,2017,6,1,2017,7,1,2017,9,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)
temp_x,temp_y=super_data(train,2017,6,1,2017,11,1,2017,12,1,2018,2,1)
train_x=train_x.append(temp_x)
train_y=np.append(train_y,temp_y)

df_val_y=train.drop(['visitStartTime'],axis=1).loc[val_y_index].groupby('fullVisitorId')['totals.totalTransactionRevenue'].sum().reset_index()
df_val_y=df_val_y.rename(columns={'totals.totalTransactionRevenue':'Revenue'})
df_val_x=grouping_id(grouping_date(train.loc[val_x_index],2018,8,1))
val_y=pd.merge(df_val_x,df_val_y,how='left',on='fullVisitorId')
val_y=np.log1p(val_y['Revenue'].fillna(0).values)
val_id=visitid[val_x_index]
val_x=df_val_x.drop(['fullVisitorId'],axis=1)
result=pd.DataFrame(columns=['iteration','learning_rate','mean'])
for itr in [5,10,20,30,40,50]:
    for rate in [0.01]:
    # for estim in [20,40,50,60,70,90,110,130,170]:
    #     average=[]
        result.loc[len(result)]=[itr,rate,run_cat2(train_x,train_y,val_x,val_y,val_id,itr,rate)]

In [58]:
#pd.DataFrame(list(zip(model.feature_importances_,train_x.columns.values)))

0.05435599257575876

,0,1
0,36.054319,totals.totalTransactionRevenue_sum
1,3.610358,totals.totalTransactionCount_sum
2,1.917608,totals.totalTransactionCount_mean
3,0.041738,totals.hits_count
4,9.830850,totals.pageviews_sum
5,3.002297,totals.pageviews_median
6,0.830267,totals.bounces_sum
7,0.000000,totals.newVisits_sum
8,0.000000,totals.newVisits_mean
9,0.000000,customDimensions_value_max
